In [5]:
!pip install qiskit-ibm-provider

In [19]:
#'13d9540e280b61b7a4254fcdb05516180957df07523dfef1924f7d9363699a51826b42e7a063b3fb685f295f6d27f8ff61810a97f503a704c07bc959d6cc6e3f'
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import QiskitRuntimeService
import json
import os
import pickle
import datetime

# 初始化並登入（第一次會自動儲存 token）
provider = IBMProvider(token='13d9540e280b61b7a4254fcdb05516180957df07523dfef1924f7d9363699a51826b42e7a063b3fb685f295f6d27f8ff61810a97f503a704c07bc959d6cc6e3f')
instances = provider.instances()
print("🧾 可用的 hub/group/project 組合：")
for inst in instances:
    print(f" - {inst}")

🧾 可用的 hub/group/project 組合：
 - ibm-q-hub-ntu/ntu-internal/default
 - ibm-q-hub-ntu/chen-hong-bin/default
 - ibm-q/open/main


In [14]:
def default_json_serializer(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

main_folder = "ibm_all_jobs_backup"
os.makedirs(main_folder, exist_ok=True)

provider = IBMProvider()
instances = provider.instances()

for instance in instances:
    hub, group, project = instance.split('/')
    print(f"\n🔍 切換到 instance: {instance}")

    # 切換 instance
    sub_provider = IBMProvider(instance=instance)

    # 抓所有完成的 job
    jobs = sub_provider.jobs(status='completed', limit=10000)

    for job in jobs:
        try:
            job_id = job.job_id
            backend_name = job.backend().name
            result = job.result()
            result_dict = result.to_dict()

            # 建資料夾（依照 instance 分類）
            job_folder = os.path.join(main_folder, f"{hub}_{group}_{project}", f"job_{job_id}")
            os.makedirs(job_folder, exist_ok=True)

            # 儲存 JSON
            with open(os.path.join(job_folder, "result.json"), "w") as f_json:
                json.dump(result_dict, f_json, indent=2, default=default_json_serializer)

            # 儲存 pickle（其實是 dict）
            with open(os.path.join(job_folder, "result.pkl"), "wb") as f_pickle:
                pickle.dump(result_dict, f_pickle)

            print(f"✅ [{backend_name}] Job {job_id} 備份完成")

        except Exception as e:
            print(f"⚠️ Job 備份失敗：{e}")



🔍 切換到 instance: ibm-q-hub-ntu/chen-hong-bin/default

🔍 切換到 instance: ibm-q-hub-ntu/ntu-internal/default
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst241gzsqjg008t9ye0')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst240rah4b0008cx1qg')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst24007ynng008yh7p0')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23zfvkv50008gd8q0')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23yfzsqjg008t9yd0')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23xqjkdzg00889960')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23wzzsqjg008t9ycg')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23w77ynng008yh7mg')>> 備份完成
✅ [ibm_osaka] Job <bound method IBMCircuitJob.job_id of <IBMCircuitJob('cst23vf

In [24]:
main_folder = "ibm_runtime_jobs_backup"
os.makedirs(main_folder, exist_ok=True)

# 初始化服務（記得填你的 instance）
service = QiskitRuntimeService(channel="ibm_quantum", instance="ibm-q-hub-ntu/chen-hong-bin/default")

jobs = service.jobs(limit=1000)

for job in jobs:
    if job.status().name != "COMPLETED":
        continue  # 自動跳過非完成的 job

    try:
        job_id = job.job_id
        backend = job.backend()
        result = job.result()
        result_dict = result if isinstance(result, dict) else result.to_dict()
        inputs = job.inputs
        program_id = job.program_id

        job_folder = os.path.join(main_folder, f"job_{job_id}")
        os.makedirs(job_folder, exist_ok=True)

        with open(os.path.join(job_folder, "result.json"), "w") as f_json:
            json.dump(result_dict, f_json, indent=2, default=default_json_serializer)

        with open(os.path.join(job_folder, "inputs.json"), "w") as f_in:
            json.dump(inputs, f_in, indent=2, default=default_json_serializer)

        with open(os.path.join(job_folder, "result.pkl"), "wb") as f_pickle:
            pickle.dump(result_dict, f_pickle)

        print(f"✅ Job {job_id} 備份完成 | Backend: {backend} | Program: {program_id}")

    except Exception as e:
        print(f"⚠️ Job {job.job_id} 備份失敗：{e}")


/tmp/ipykernel_2853/475209082.py:10: DeprecationWarning: In a future release of qiskit-ibm-runtime no sooner than 3 months after the release date of 0.30.0, RuntimeJob.status() will be returned as a string instead of an instance of `JobStatus`. To prepare for this change, you can use the idiom `status.name if isinstance(status, JobStatus) else status`.
  if job.status().name != "COMPLETED":


In [21]:
!pip install --upgrade qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 84.5 MB/s eta 0:00:00
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.2.0
    Uninstalling qiskit-1.2.0:
      Successfully uninstalled qiskit-1.2.0
  Attempting uninstall: qiskit-ibm-runtime
    Found existing installation: qiskit-ibm-runtime 0.34.0
    Uninstalling qiskit-ibm-runtime-0.34.0:
      Successfully uninstalled qiskit-ibm-runtime-0.34.0
